In [8]:
#Importing configuration
import configparser
config = configparser.ConfigParser()
config.read("config.txt")

word2vec_path = config.get("configuration","word2vec_path")
stanford_corenlp_path = config.get("configuration","stanford_corenlp_path")


import pymysql
pymysql.install_as_MySQLdb()

# Connect
mydbhost = config.get("configuration","mydbhost")
mydbuser = config.get("configuration","mydbuser")
mydbpasswd = config.get("configuration","mydbpasswd")
mydbdb = config.get("configuration","mydbdb")
connection = pymysql.connect(host=mydbhost,
                             user=mydbuser,
                             passwd=mydbpasswd,
                             db=mydbdb)

#print ("connect successful!!")

text = "A hacker gained access to the PHI for the covered entity's patients."
 
try:
    with connection.cursor() as cursor:
        # Execute SQL select statement
        cursor.execute("SELECT instance, class FROM simple_types where instance like '%Munindar%'")
        # Commit your changes if writing
        # In this case, we are only reading data
        # db.commit()
        
        # Get the number of rows in the resultset
        numrows = cursor.rowcount
        
        # Get and display one row at a time
        for x in range(0, numrows):
            row = cursor.fetchone()
            print(row[0], "-->", row[1])
            
# Close the connection
finally:
    # Close connection.
    connection.close()

<Munindar_P._Singh> --> <wordnet_scientist_110560637>
<Munindar_P._Singh> --> <wikicat_Artificial_intelligence_researchers>
<Munindar_P._Singh> --> <wikicat_Living_people>
<Munindar_P._Singh> --> <wikicat_Computer_scientists>


In [9]:
from stanfordcorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP(stanford_corenlp_path)
connection = pymysql.connect(host=mydbhost,
                             user=mydbuser,
                             passwd=mydbpasswd,
                             db=mydbdb)

try: 
    with connection.cursor() as cursor:
        #cursor.execute("Select text, day, place, attacker, target, instrument from cyberattacks")
        cursor.execute("Select 'ISIS members threw stones in Paris.'")
        numrows = cursor.rowcount
        for x in range(0, numrows):
            row = cursor.fetchone()
            sentence = row[0]

            print('Sentence:', sentence)
            print('Tokenize:', nlp.word_tokenize(sentence))
            print('Part of Speech:', nlp.pos_tag(sentence))
            print('Named Entities:', nlp.ner(sentence))
            print('Constituency Parsing:', nlp.parse(sentence))
            print('Dependency Parsing:', nlp.dependency_parse(sentence))
finally: 
    connection.close()
    nlp.close() # Do not forget to close! The backend server will consume a lot memery.

Sentence: ISIS members threw stones in Paris.
Tokenize: ['ISIS', 'members', 'threw', 'stones', 'in', 'Paris', '.']
Part of Speech: [('ISIS', 'NNP'), ('members', 'NNS'), ('threw', 'VBD'), ('stones', 'NNS'), ('in', 'IN'), ('Paris', 'NNP'), ('.', '.')]
Named Entities: [('ISIS', 'ORGANIZATION'), ('members', 'O'), ('threw', 'O'), ('stones', 'O'), ('in', 'O'), ('Paris', 'CITY'), ('.', 'O')]
Constituency Parsing: (ROOT
  (S
    (NP (NNP ISIS) (NNS members))
    (VP (VBD threw)
      (NP (NNS stones))
      (PP (IN in)
        (NP (NNP Paris))))
    (. .)))
Dependency Parsing: [('ROOT', 0, 3), ('compound', 2, 1), ('nsubj', 3, 2), ('dobj', 3, 4), ('case', 6, 5), ('nmod', 3, 6), ('punct', 3, 7)]


In [13]:
# importing StandfordCoreNLP to tokenize, tag, and ner
nlp = StanfordCoreNLP(stanford_corenlp_path)

sentence = "ISIS members threw stones in Paris."
sentence_tokens = nlp.word_tokenize(sentence)
sentence_tags = nlp.pos_tag(sentence)
sentence_ner = nlp.ner(sentence)
#sentence_parse = nlp.parse(sentence)
#sentence_dependency = nlp.dependency_parse(sentence)

to_replace_ners = []
to_replace_verbs = []

for (i, j) in sentence_ner:
    #print(i, j)
    if(j!='O'):
        print(i, j)
        to_replace_ners.append((i,j))
        
verb_check = 0
        
for (i, j) in sentence_tags:
    if(verb_check == 1):
        verb = verb + '_' + i
        to_replace_verbs.append((verb, i))
        verb_check = 0
        print(verb)
    
    if(j=='VBD'):
        print(i, j)
        verb_check = 1
        verb = i
    
        
nlp.close()

ISIS ORGANIZATION
Paris CITY
threw VBD
threw_stones


In [18]:
# Importing word2vec to find similarity and neighboring words
import gensim
from gensim.models import Word2Vec

model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True) 

In [20]:
topk = 10
replacement_ners = []
replacement_verbs = []

for (i, j) in to_replace_ners:
    similar_ners = model.most_similar(i, [], topk)
    replacement_ners.append((i, similar_ners))

print(replacement_ners)
    
for (verb, nn) in replacement_verbs:
    similar_verbs = model.most_similar(verb, [], topk)
    replacement_verbs.append((verb,similar_verbs))

print(replacement_verbs)

[('ISIS', [('Isis_Pharmaceuticals_NASDAQ', 0.5358301997184753), ('stratospheric_airship', 0.4330335855484009), ('SILEX', 0.42576032876968384), ('DECIPHER', 0.4255940914154053), ('Isis_Pharmaceuticals_Inc', 0.42540621757507324), ('Corey_Hinderstein', 0.4233326017856598), ('3D_QSAR', 0.417447030544281), ('STFC_Daresbury_Laboratory', 0.41502732038497925), ('Agilent_SureSelect', 0.41367751359939575), ('Excaliard', 0.4103640615940094)]), ('Paris', [('Parisian', 0.6789354085922241), ('Hopital_Europeen_Georges_Pompidou', 0.6536555290222168), ('Spyker_D##_Peking', 0.6336591839790344), ('France', 0.6334909796714783), ('Pantheon_Sorbonne', 0.6312518119812012), ('Aeroports_De', 0.621803879737854), ('Grigny_south', 0.6194689273834229), ('Place_Denfert_Rochereau', 0.6028153896331787), ('guest_Olivier_Dolige', 0.6024351119995117), ('Lazard_Freres_Banque', 0.5998712778091431)])]
[]


In [55]:
model.most_similar(['Cornell', 'Shanghai'], ['New_York'], 100)

[('Fudan', 0.5536324381828308),
 ('Xu', 0.5300264358520508),
 ('Tsinghua', 0.5270693302154541),
 ('Zhu', 0.5268391370773315),
 ('Zhang', 0.5170268416404724),
 ('Xie', 0.5124308466911316),
 ('Peking_University', 0.5123456716537476),
 ('SJTU', 0.5045580863952637),
 ('Jiao_Tong', 0.501052975654602),
 ('Feng', 0.4988190531730652),
 ('Lu', 0.49630722403526306),
 ('Qiu', 0.4937359094619751),
 ('Guo', 0.493697851896286),
 ('Tian', 0.4930003881454468),
 ('Donghua_University', 0.4914301037788391),
 ('Tongji', 0.4912794828414917),
 ('Li', 0.48988276720046997),
 ('Tsinghua_University', 0.48851269483566284),
 ('Shangguan', 0.48774391412734985),
 ('Xiao', 0.48583245277404785),
 ('Weiming', 0.4857281744480133),
 ('Jiao_Tong_University', 0.48433077335357666),
 ('prestigious_Fudan', 0.4840565323829651),
 ('Ouyang', 0.48237305879592896),
 ('Nankai_University', 0.47940143942832947),
 ('Xin', 0.4777411222457886),
 ('Shenzhen', 0.47725600004196167),
 ('PolyU', 0.476238876581192),
 ('Nottingham_Ningbo', 0.